### **Summary**
In this notebook I loaded the dataset from .mat file,
the data is a LARGE photo with 6-Channels.

---

* 1st Channel is Blue.
* 2nd Channel is Green. 
* 3rd Channel is Red.
* 4th, 5th and 6th are Infrared.

In [ ]:
from scipy.io import loadmat
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
!pip install patchify
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from tensorflow.keras.utils import to_categorical
patch_size = 256
file_path = '/content/drive/MyDrive/DS-LAB1/rit18_data.mat'
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# file_path = '/content/drive/MyDrive/DS-LAB1/rit18_data.mat'
# dataset = loadmat(file_path)

# #Load Training Data and Labels
# train_data = dataset['train_data']
# train_mask = train_data[-1]
# train_data = train_data[:6]
# train_labels = dataset['train_labels']

# #Load Validation Data and Labels
# val_data = dataset['val_data']
# val_mask = val_data[-1]
# val_data = val_data[:6]
# val_labels = dataset['val_labels']

# #Load Test Data
# test_data = dataset['test_data']
# test_mask = test_data[-1]
# test_data = test_data[:6]

# band_centers = dataset['band_centers'][0]
# band_center_units = dataset['band_center_units']
# classes = dataset['classes']                          

# print("Train data shape", train_data.shape)
# print("Validation data shape", val_data.shape)
# print("Test data shape", test_mask.shape)

In [ ]:
def reshape_data(data):
  return np.rollaxis(data,0,3)

def data2patches(data, shape: tuple, patch_size, scaler = None):
  dataset = []
  # ------- TO DO ------- solve the reminder size/patch_size pixel issue 
  # shape is like (patch_size, patch_size, num_of_channels)

  patches_img = patchify(data, shape, step=patch_size)

  for i in range(patches_img.shape[0]):
      for j in range(patches_img.shape[1]):

          
          single_patch_img = patches_img[i,j,:,:]
          
          if scaler: 
            single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape)
            single_patch_img = single_patch_img[0] #Drop the extra unecessary dimension that patchify adds, for data only not for masks       
            single_patch_img = (single_patch_img.astype('float32')) / 255.       

          dataset.append(single_patch_img)
  return np.array(dataset)
  
def save_npy(npdata, filename):
  np.save(f'/content/drive/MyDrive/DS-LAB1/{filename}',npdata)

In [ ]:
# 1. Load train data and it's labels
dataset = loadmat(file_path)
train = dataset['train_data']
train_labels = dataset['train_labels']
train_data = train[:6]
del train, dataset
gc.collect()

212

In [ ]:
# 2. Reshape the data, generate patches and save it to file.
train = reshape_data(train_data)
del train_data
dataset = data2patches(train, (patch_size,patch_size,6),patch_size,scaler)
del train
save_npy(dataset,'train_data')
del dataset
gc.collect()

50

In [ ]:
# 3. generate patches from train labels, generate one-hot encoding 
#    and save it to file.
temp = data2patches(train_labels, (patch_size,patch_size),patch_size)
train_labels_cat = to_categorical(temp, num_classes=19)
print(train_labels_cat.shape)
save_npy(train_labels_cat,'train_labels')
del temp, train_labels, train_labels_cat
gc.collect()

(792, 256, 256, 19)


50

In [ ]:
# 1. Load val data and it's labels
dataset = loadmat(file_path)
val = dataset['val_data']
val_labels = dataset['val_labels']
val_data = val[:6]
del val, dataset
gc.collect()

52

In [ ]:
# 2. Reshape the data, generate patches and save it to file.
val = reshape_data(val_data)
del val_data
dataset = data2patches(val, (patch_size,patch_size,6),patch_size,scaler)
del val
save_npy(dataset,'val_data')
del dataset
gc.collect()

50

In [ ]:
# 3. generate patches from train labels, generate one-hot encoding 
#    and save it to file.
temp = data2patches(val_labels, (patch_size,patch_size),patch_size)
val_labels_cat = to_categorical(temp, num_classes=19)
del temp
print(val_labels_cat.shape)
save_npy(val_labels_cat,'val_labels')
del val_labels, val_labels_cat
gc.collect()

(918, 256, 256, 19)


50